In [ ]:
import pandas as pd
import numpy as np

import warnings
import itertools
warnings.filterwarnings("ignore")
import statsmodels.api as sm
import requests
import pandas as pd
import datetime
import pytz
from skimage import io
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
import os

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Input, Flatten
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, BatchNormalization, Flatten, Dropout, Dense, Conv1D, MaxPooling1D, BatchNormalization, GRU
import joblib 

In [2]:
import os

# Create a folder for the experiment files
experiment_folder = 'sensor_datastore'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created.')

sensor_datastore folder created.


In [ ]:
# %%writefile Helper/__init__.py

 

In [3]:
os.chdir("/mnt/batch/tasks/shared/LS_root/mounts/clusters/test123243/code/Time Series")

NameError: name 'ws' is not defined

In [ ]:
#extra

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
sensor_env = Environment("sensor-experiment-env")
sensor_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
sensor_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies (conda or pip as required)
sensor_packages = CondaDependencies.create(conda_packages=[],
                                          pip_packages=['azureml-widgets', 'azureml-sdk', "joblib", 'scikit-image', 'tensorflow', 'pandas', 'statsmodels'])

# Add the dependencies to the environment
sensor_env.python.conda_dependencies = sensor_packages

print(sensor_env.name, 'defined.')
sensor_env.register(workspace=ws)



# Save the environment config as a .yml file
folder_name = "outputs"
env_file = folder_name + "/conda_env.yml"

with open(env_file,"w") as f:
    f.write(sensor_env.python.conda_dependencies.serialize_to_string())

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

In [ ]:
# Save the environment config as a .yml file
env_file = folder_name + "/sensor_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

In [ ]:
from data_processing import dataProcessing
from azureml.core import Workspace

In [5]:
#
# Azure ml
# 
ws = Workspace.get(name='MLN',
                   subscription_id='badcc2bf-a4d3-4b35-9d81-5bc289cd48e3',
                   resource_group='MLR')

for ct_name in ws.compute_targets:
    print(ct_name)
    
    
experiment_folder = 'sensor_datastore'

default_ds = ws.get_default_datastore()

NameError: name 'Workspace' is not defined

In [7]:

from azureml.train.estimator import Estimator
from azureml.core import Environment, Experiment
from azureml.widgets import RunDetails
from azureml.core import Dataset
from azureml.core import Workspace

#
# Azure ml
# 

ws = Workspace.get(name='MLN',
                   subscription_id='badcc2bf-a4d3-4b35-9d81-5bc289cd48e3',
                   resource_group='MLR')

for ct_name in ws.compute_targets:
    print(ct_name)
    
    
experiment_folder = 'sensor_datastore'

default_ds = ws.get_default_datastore()


# Get the environment
registered_env = Environment.get(ws, 'sensor-experiment-env')

# Set the script parameters
script_params = {'--order': '(0, 0, 1)', '--seasonal_order': '(0, 0, 1, 24)'}

# Get the training dataset
sensor_all = Dataset.Tabular.from_delimited_files(path=(default_ds, 'sensor-data/sensor_all.csv'))

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
                      inputs=[sensor_all.as_named_input('sensor_all')],
                      script_params = script_params,
                      compute_target = ct_name, # Run the experiment on the remote compute target
                      environment_definition = registered_env,
                      entry_script='./sensor_training.py')

# Create an experiment
experiment = Experiment(workspace = ws, name = 'sensor-training')

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

test4563345
test123243


KeyboardInterrupt: 

In [ ]:
%%writefile sensor_training.py
# Import libraries
from azureml.core import Run
import argparse
import pandas as pd
import joblib
import statsmodels.api as sm

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument('--order', type=str, dest='order', default=(0,0,1))
parser.add_argument('--seasonal_order', type=str, dest='seasonal_order', default=(0,0,1,24))
args = parser.parse_args()
order = eval(args.order)
seasonal_order = eval(args.seasonal_order)

# Get the experiment run context
run = Run.get_context()

# Get sensor data
sensor_all = run.input_datasets['sensor_all'].to_pandas_dataframe()

# Prepare for training 
exog_data  = sensor_all[["value_one", "value_two", "month", "datetime"]].set_index('datetime')
endog = sensor_all[["target", "datetime"]].set_index('datetime') 

# Model
mod = sm.tsa.statespace.SARIMAX(endog= endog,
                                exog = exog_data,
                                order=order,
                                seasonal_order=seasonal_order,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print('SARIMA{}x{} - AIC:{}'.format(order, seasonal_order, results.aic))


results.save('model.pkl')
run.upload_file(name = 'outputs/model.pkl', path_or_stream = './model.pkl')

# Register the model
run.register_model(model_path='outputs/model.pkl', model_name='sensor_model_1',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': results.aic})


# Log metrics
run.log('seasonal_order', str(seasonal_order))
run.log('order', str(order))
run.log('aic', str(results.aic))


run.complete()

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))

In [ ]:

sensor_all = Dataset.Tabular.from_delimited_files(path=(default_ds, 'sensor-data/sensor_all.csv')).to_pandas_dataframe()
# Prepare for training 
exog_data  = sensor_all[["value_one", "value_two", "month", "datetime"]].set_index('datetime')
endog = sensor_all[["target", "datetime"]].set_index('datetime') 

mod = sm.tsa.statespace.SARIMAX(endog= endog,
                                exog = exog_data,
                                order=(0,0,1),
                                seasonal_order=(0,0,1,24),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print('SARIMA{}x{} - AIC:{}'.format(order, seasonal_order, results.aic))

In [ ]:
%%writefile $experiment_folder/prediction_sensor.py
import json
import joblib
import numpy as np
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('outputs/model.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    predictions = model.forecast(exog=data)
    return json.dumps({"Time":str(predictions.index[0]), "prediction": predictions[0]})

In [ ]:
import json
raw_data = json.dumps({"data": [17.3     , 11.373132,  1.      ]})

In [ ]:
run(raw_data)

In [9]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

sensor_model_1 version: 3
	 Training context : Inline Training
	 AUC : 34057.26026468643


sensor_model_1 version: 2
	 Training context : Inline Training
	 AUC : 34057.26026468643


sensor_model_1 version: 1
	 Training context : Inline Training
	 AUC : 34057.26026468643


sensor_model version: 2
	 Training context : Inline Training
	 AUC : 34057.26026468643


sensor_model version: 1
	 Training context : Inline Training
	 AUC : 34057.26026468643




In [11]:
%%writefile deployment_sensor.py
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

service_name = "sensor-service5"

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/test123243/code/Time Series/',
                                   entry_script="./sensor_datastore/prediction_sensor.py",
                                   conda_file="./outputs/conda_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)


Writing deployment_sensor.py


In [ ]:
service.run(raw_data)

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

In [ ]:
service.state

In [ ]:
!pwd